In [1]:
import graph_tool.all as gt                       # Biblioteca para GRAFO
import csv  
import math

In [2]:
class VisitorExample(gt.AStarVisitor):                                            # É um objeto visitante que é chamado nos pontos de evento dentro do algoritmo bfs_search()
    def __init__(self, name, time, name_time, v_color, dist, pred, e_color, e_action, e_ord, target):
        self.name = name
        self.time = time
        self.name_time = name_time
        self.fill_color = v_color
        self.dist = dist
        self.pred = pred
        self.color = e_color
        self.e_action = e_action
        self.e_ord = e_ord
        self.e_count = 0
        self.last_time = 0
        self.target = target
        
    def discover_vertex(self, u):                                               # Invocado quando um vértice é encontrado pela primeira vez.
        self.name[u] = v_name[u]
        self.time[u] = self.last_time
        self.last_time += 1        
        self.name_time[u] = str(self.name[u]) + "(" + str(self.time[u]) + ")"
        #print("-->", self.name[u], "foi encontrado e entrou na FILA") 
        self.fill_color[u] = "white"

    def examine_vertex(self, u):                                                # Invocado em um vértice à medida que é retirado da fila. 
        #print("   ",self.name[u], "saiu da FILA e está sendo analisado (expandido)...") 
        return

    def examine_edge(self, e):                                                  # Invocado em cada aresta de cada vértice depois de descoberto.
        #print("    Aresta (%s, %s) em análise..." %               (v_name[e.source()], v_name[e.target()]))
        return
    def edge_relaxed(self, e):                                                  #Após o exame do vértices, este método é invocado. 
        
        self.pred[e.target()] = int(e.source())                                 
        self.dist[e.target()] = self.dist[e.source()] + 1
        e = g_astar.add_edge(int(e.source()), int(e.target()))
        self.color[e] = "gray"
        self.e_action[e] = e_action[g.edge(int(e.source()), int(e.target()))]
        self.e_count += 1
        self.e_ord[e] = self.e_count
        #print("    Após análise, aresta (%s, %s) mantida." %             (v_name[e.source()], v_name[e.target()]))
       
    def finish_vertex(self, u):
        #print("    > Todos os vértices adjacentes à", self.name[u], "foram descobertos!\n") 
        return
      

In [3]:
g = gt.Graph() 
g.set_directed(False)                             # criação do objeto
v_name = g.new_vertex_property("string")          # referenciação da lista v_name com uma nova propriedade (label) criada para o vértice - tipo string  
v_pos  = g.new_vertex_property("vector<double>")
e_distance = g.new_edge_property("double") 
e_weight = g.new_edge_property("float")  
e_action = g.new_edge_property("string") 



f_network = open("vertices.csv", 'r', encoding='utf-8')
reader_network = csv.reader(f_network, delimiter=",")

vposX = []
vposY = []
heuristic = [] 
for vertice in reader_network:
    v = g.add_vertex()
    v_name[v] = str(vertice[1])
    vposX.append(vertice[5]) 
    vposY.append(vertice[6]) 
f_network.close()


f_network = open("arestas.csv", 'r', encoding='utf-8')
reader_network = csv.reader(f_network, delimiter=",")
for edge in reader_network:
    e = g.add_edge(int(edge[0]), int(edge[1]))
    e_weight[e] = float(edge[2])   
    e_distance[e] = float(edge[3])
   
f_network.close()

for v in g.vertices():
    v_pos[v] = (vposX[int(v)],vposY[int(v)])

In [5]:
for index_Destiny in range(0,61,1):
    for index_Source in range(0,61,1):
        if index_Source == index_Destiny:
            continue
        def DistanciaEuclidiana(v, target):
            valor = (float(vposX[v]) - float(vposX[target]))**2 + (float(vposY[v]) - float(vposY[target]))**2
            return math.sqrt(valor)

        def PrenncherHeuristica(target):   
            heuristic = []
            for v in v_name:         
                index_atual = list(v_name).index(v)   
                heuristic.append(DistanciaEuclidiana(index_atual, target))
            return heuristic 


        heuristic = PrenncherHeuristica(index_Destiny) #erro estava no valor de destino, sempre que
                                #muda o destino trocar esse valor do parametro
        def he(v): 
            return heuristic[v]


        g_astar = gt.Graph()                                    # criação do objeto para busca em Largura
        v_name_dfs = g_astar.new_vertex_property("string")      # referenciação da lista v_name com uma nova propriedade (label) criada para o vértice - tipo string 
        e_ord = g_astar.new_edge_property("int")                # referenciação da lista e_ord com uma nova propriedade criada para a ordem de expansão - tipo int
        e_action_bfs = g_astar.new_edge_property("string")

        f_network = open("vertices.csv", 'r', encoding='utf-8')
        reader_network = csv.reader(f_network, delimiter=",")
        for vertice in reader_network:
            v = g_astar.add_vertex()
            v_name_dfs[v] = str(vertice[1])

        f_network.close()


        index_raiz = index_Source
        ord = 1


        for edge in gt.astar_iterator(g, g.vertex(index_raiz), e_distance, heuristic=lambda v: he(int(v))):
           e = g_astar.add_edge(int(edge.source()), int(edge.target()))
           e_ord[e] = ord
           e_action_bfs[e] = '(' + str(ord) + ') ' #+ e_action[g.edge(int(edge.source()), int(edge.target()))] 
           ord += 1


        g_astar = gt.Graph()                                      # criação do objeto para busca A Estrela
        bfsv_name = g_astar.new_vertex_property("string")      # referenciação da lista v_name_bfs com uma nova propriedade do vértice para o nome - tipo string 
        bfsv_time = g_astar.new_vertex_property("int")         # referenciação da lista v_time com uma nova propriedade do vértice para a ordem de expansão - tipo int
        bfsv_name_time = g_astar.new_vertex_property("string")      # referenciação da lista v_name_time com uma nova propriedade do vértice para o nome e ordem de expansão - tipo string
        bfsv_color = g_astar.new_vertex_property("string")      # referenciação da lista v_color com uma nova propriedade do vértice para a cor - tipo string  
        bfsv_dist = g_astar.new_vertex_property("int")         # referenciação da lista v_dist como uma propriedade do vértice criada para a distância da raiz
        bfsv_pred = g_astar.new_vertex_property("int64_t")     # referenciação da lista v_pred como uma propriedade do vértice para referenciar o predecessor (pai)
        bfse_color = g_astar.new_edge_property("string")        # referenciação da lista e_color com uma nova propriedade da aresta para a cor - tipo string  
        bfse_action = g_astar.new_edge_property("string")        # referenciação da lista e_action_bfs com uma nova propriedade da aresta para a ação - tipo string
        bfse_ord = g_astar.new_edge_property("string")        # referenciação da lista e_action_bfs com uma nova propriedade da aresta para a ação - tipo string
        bfse_weight_bfs = g_astar.new_edge_property("float")       # referenciação da lista e_weight com uma nova propriedade criada para a função g(n) - tipo float
        v_pos  = g.new_vertex_property("vector<double>")



        index_raiz = index_Source
        index_alvo = index_Destiny


        gt.astar_search(g, g.vertex(index_Source), e_distance, VisitorExample(bfsv_name, bfsv_time, bfsv_name_time, bfsv_color, 
                                            bfsv_dist, bfsv_pred, bfse_color, bfse_action, bfse_ord, g.vertex(index_Destiny)), heuristic=lambda v: he(int(v))) #, heuristic=lambda v: h(index_raiz, index_alvo)


        bfsv_color[index_alvo] = "green"
        path = []
        id_caminho = []
        id_caminho.insert(0, index_alvo)

        while index_alvo != index_raiz:
          e = g_astar.edge(bfsv_pred[index_alvo], index_alvo)
          if e is None:
            break;
          bfse_color[e] = "red"
          index_alvo = bfsv_pred[index_alvo]
          path.insert(0,bfsv_name[index_alvo])
          id_caminho.insert(0, index_alvo)
          bfsv_color[index_alvo] = "#729fcf"

        path = [str(id) for id in id_caminho]
        path = ",".join(path)
        _file = open(f"table_q_AStar/table_q_{index_Destiny}.csv", "a", encoding="utf-8")  
        _file.write(f"{path}\n")
        _file.close()
        #_file = open(f"table_q_astar/delta_q_{index_Destiny}.csv", "a", encoding="utf-8")
        #_file.write(f"{origem},{destino}\n")
        #_file.close()
